## Imports 

In [2]:
import common.feature_string as feature_string
import common.feature_num as feature_num
import common.feature_categorica as feature_categorica
import common.common_machine_learning as common
import common.my_pipeline as my_pipe
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import time
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score

### Ignore Warnings 

In [3]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## Constantes

## Set de datos 

In [4]:
train = common.cargar_set_optimizado('sets_de_datos/train.csv', index_col = 0)

In [5]:
train.columns

Index(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

Analizar nuevas features:  
* Precio segun año => Como año
* Precio segun mes => OHE
* Precio segun provincia con mas de 1000 registros => OHE
* Precio segun ciudad con mas de 1000 registros => OHE
* Precio segun lat, lng => Rotaciones
* Precio segun idzona con mas de 
* Cero Garages, habitaciones, baños y no es Casa o Apartamento  
  
Extraer y generar csv's para los nuevos datos de Nicky

In [31]:
train['cantidad'] = 1
train.groupby(['provincia'])['cantidad'].apply(lambda x: x.sum())

provincia
Aguascalientes            1753
Baja California Norte     3220
Baja California Sur       1700
Campeche                   263
Chiapas                   1126
Chihuahua                 4590
Coahuila                  3695
Colima                    1008
Distrito Federal         58790
Durango                   2275
Edo. de México           41607
Guanajuato                2860
Guerrero                  2678
Hidalgo                   2521
Jalisco                  21238
Michoacán                 2471
Morelos                   7337
Nayarit                   1352
Nuevo León               15324
Oaxaca                     711
Puebla                   10421
Querétaro                16988
Quintana Roo              4756
San luis Potosí           8447
Sinaloa                   1806
Sonora                    2988
Tabasco                    994
Tamaulipas                2303
Tlaxcala                   839
Veracruz                  5762
Yucatán                   7928
Zacatecas                   9